# Iniciando Sessão Spark

In [1]:
from pyspark.sql import SparkSession

# Iniciar a sessão Spark com a configuração do Delta Lake
spark = (SparkSession.builder
    .appName("DeltaLake")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Extensão do Delta
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Catálogo do Delta
    .config("spark.jars", "/usr/local/spark/jars/delta-spark_2.12-3.0.0.jar,/usr/local/spark/jars/delta-storage-3.0.0.jar")  # Caminho dos JARs no Docker
    .getOrCreate()) # Se utilizar as limitações de hardware pelo spark, mover para o final
#    .config("spark.executor.cores", "2")  # Limitar o número de núcleos por executor
#    .config("spark.driver.cores", "1")    # Limitar o número de núcleos para o driver
#    .config("spark.executor.memory", "4g")
#    .config("spark.driver.memory", "4g")
#    .config("spark.memory.fraction", "0.3")  # Ajuste para controlar a fração de memória usada pelo Spark
#    .config("spark.memory.storageFraction", "0.2")  # Fração da memória usada para armazenar dados persistentes"""

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import udf

import time

# Verificar se a sessão Spark foi criada com sucesso
print(spark.version)

3.5.0


# Importando CSV

In [ ]:
df_test = spark.read.csv("/home/jovyan/app/data/raw/temas_ambientais.csv", sep=";", header=True)
df_test.show()

# Salvando Bronze (Distribuída)

In [ ]:
df_test = df_test.dropDuplicates()

In [ ]:
df_test.write.mode("overwrite").format("delta").save("/home/jovyan/app/data/bronze/bronze_distribuida")

# Limpeza e Salvando Silver (Distribuída)

In [ ]:
df_test = spark.read.format("delta").load("/home/jovyan/app/data/bronze/bronze_distribuida")
df_test.show(10)

In [ ]:
# Criar uma lista com o nome das colunas e o número de nulos em cada uma
null_counts_1 = df_test.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_test.columns]).first().asDict()

# Exibir o resultado
null_counts_1

In [ ]:
df_test.printSchema()

In [ ]:
int_columns = ["codigo_ibge"]
float_columns = ["area_do_imovel", "area_liquida", "area_remanescente_vegetacao_nativa", 
                 "area_reserva_legal_proposta", "area_preservacao_permanente", "area_nao_classificada",
                 "latitude", "longitude", "area_rural_consolidada", "area_servidao_administrativa",
                 "modulos_fiscais", "area_uso_restrito", "area_reserva_legal_averbada",
                 "area_reserva_legal_aprovada_nao_averbada", "area_pousio"]
bool_columns = ["solicitacao_adesao_pra"]
date_columns = ["data_inscricao", "data_alteracao_condicao_cadastro", "data_ultima_retificacao"]
text_column = [x for x in df_test.columns if x not in int_columns+float_columns+bool_columns+date_columns]

In [ ]:
# Dados Para Integer
df_test = df_test.withColumn("codigo_ibge", col("codigo_ibge").cast(IntegerType()))

# Dados Para Float
for i in float_columns:
    df_test = df_test.withColumn(i, col(i).cast(FloatType()))

# Dados para Bool
df_test = df_test.withColumn("solicitacao_adesao_pra", 
                             when(col("solicitacao_adesao_pra") == "Sim", True).
                             when(col("solicitacao_adesao_pra") == "Nao", False).
                             otherwise(None))

# Dados para Date
for i in date_columns:
    df_test = df_test.withColumn(i, to_timestamp(regexp_replace(col(i), r"\.\d+$", ""), "yyyy-MM-dd HH:mm:ss"))

df_test.printSchema()

In [ ]:
# Criar uma lista com o nome das colunas e o número de nulos em cada uma
null_counts_2 = df_test.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_test.columns]).first().asDict()

# Exibir o resultado
null_counts_2

In [ ]:
if null_counts_1 == null_counts_2:
    print("Os valores de nulos são os mesmos em ambas as versões.")
else:
    print("Há diferenças nos valores de nulos.")
    # Mostrar as diferenças
    for coluna in null_counts_1.keys():
        if null_counts_1[coluna] != null_counts_2[coluna]:
            print(f"Coluna '{coluna}': null_counts_1 = {null_counts_1[coluna]}, null_counts_2 = {null_counts_2[coluna]}")

# Salvando Silver (Distribuída - Por Estado)

In [ ]:
(df_test.write.mode("overwrite")
              .format("delta")
              .partitionBy("uf")
              .save("/home/jovyan/app/data/silver/silver_distribuida_uf"))

# Funções e Definições

In [3]:
def delta_table_query(query):
    query = spark.sql(query)
    
    query.show()

def get_time(function, query):
    start_time = time.time()
    
    function(query)

    end_time = time.time()

    tempo_query = end_time - start_time

    return tempo_query

def is_point_in_path(x: float, y: float, poly: list[tuple[float, float]]) -> bool:
    """Determine if the point is on the path, corner, or boundary of the polygon

    Args:
    x -- The x coordinates of point.
    y -- The y coordinates of point.
    poly -- a list of tuples [(x, y), (x, y), ...]

    Returns:
        True if the point is in the path or is a corner or on the boundary"""
    
    c = False

    for i in range(len(poly)):
        ax, ay = poly[i]
        bx, by = poly[i - 1]
        if (x == ax) and (y == ay):
            # point is a corner
            return True
        if (ay > y) != (by > y):
            slope = (x - ax) * (by - ay) - (bx - ax) * (y - ay)
            if slope == 0:
                # point is on boundary
                return True
            if (slope < 0) != (by < ay):
                c = not c

    return c

# Registrar a UDF no Spark
poly = [(-53.5325072, -19.4632582), (-51.0495971, -19.1625841), (-51.3734501, -16.1924262), (-53.8181518, -16.4010783), (-53.5325072, -19.4632582)]

is_point_in_polygon_udf = udf(lambda x, y: is_point_in_path(x, y, poly), BooleanType())

spark.udf.register("is_point_in_polygon", is_point_in_polygon_udf)

# Querys a partir das Delta Tables

## Consulta 1

Recupere a soma de área (em hectares) para todas as propriedades agrícolas que pertencem ao MS e MT. Ordene os resultados em ordem decrescente.

In [4]:
query = """
SELECT 
   uf,
   FORMAT_NUMBER(SUM(area_do_imovel), 3) as soma_area_hectares
FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

WHERE uf IN ('MS', 'MT')

GROUP BY uf

ORDER BY soma_area_hectares DESC

"""

tempo_delta_1 = get_time(delta_table_query, query)
tempo_delta_1

+---+------------------+
| uf|soma_area_hectares|
+---+------------------+
| MT|    84,587,089.218|
| MS|    36,321,882.660|
+---+------------------+



13.060543537139893

## Consulta 2
Filtre todas as propriedades que pertecem a região sudeste. 

In [5]:
query = """
SELECT
   uf,
   codigo_ibge

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

"""

tempo_delta_2 = get_time(delta_table_query, query)
tempo_delta_2

+---+-----------+
| uf|codigo_ibge|
+---+-----------+
| AC|    1200302|
| AC|    1200252|
| AC|    1200401|
| AC|    1200104|
| AC|    1200450|
| AC|    1200385|
| AC|    1200609|
| AC|    1200401|
| AC|    1200401|
| AC|    1200500|
| AC|    1200401|
| AC|    1200609|
| AC|    1200401|
| AC|    1200500|
| AC|    1200302|
| AC|    1200500|
| AC|    1200302|
| AC|    1200500|
| AC|    1200401|
| AC|    1200385|
+---+-----------+
only showing top 20 rows



1.6391515731811523

## Consulta 3
Recupere todas as propriedades rurais que estão localizadas dentro de uma área geográfica específica delimitada por um polígono. Este polígono é descrito pelas seguintes coordenadas: POLYGON ((-53.5325072 -19.4632582, -51.0495971 -19.1625841, -51.3734501 -16.1924262, -53.8181518 -16.4010783, -53.5325072 -19.4632582))
Verificar se um centroide está dentro dos pontos informados (ou mais).

In [6]:
query = """
SELECT
    codigo_ibge,
    latitude,
    longitude,
    is_point_in_polygon(latitude, longitude) AS inside_polygon

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

WHERE is_point_in_polygon(latitude, longitude) = True
"""

tempo_delta_3 = get_time(delta_table_query, query)
tempo_delta_3

+-----------+--------+---------+--------------+
|codigo_ibge|latitude|longitude|inside_polygon|
+-----------+--------+---------+--------------+
+-----------+--------+---------+--------------+



17.52910017967224

## Consulta 4
Calcule quantas propriedades foram cadastradas por ano. Apresente os resultados em ordem cronológica.

In [7]:
query = """
SELECT
   YEAR(data_inscricao) AS ano_inscricao,
   COUNT(*) AS qnt_ano

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

GROUP BY ano_inscricao

ORDER BY ano_inscricao
"""

tempo_delta_4 = get_time(delta_table_query, query)
tempo_delta_4

+-------------+-------+
|ano_inscricao|qnt_ano|
+-------------+-------+
|         NULL|     12|
|         2013|    743|
|         2014| 227468|
|         2015|1240176|
|         2016|1828786|
|         2017| 794777|
|         2018| 724856|
|         2019| 746570|
|         2020| 463322|
|         2021| 443942|
|         2022| 368448|
+-------------+-------+



6.882412910461426

## Consulta 5
Calcule o percentual médio de área remanescente de vegetação nativa em comparação a área total da propriedade

In [8]:
query = """
SELECT
   codigo_ibge,
   area_do_imovel,
   area_remanescente_vegetacao_nativa,
   FORMAT_NUMBER(area_remanescente_vegetacao_nativa / area_do_imovel * 100, 2) as percentual_area_nativa

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`
"""

tempo_delta_5 = get_time(delta_table_query, query)
tempo_delta_5

+-----------+--------------+----------------------------------+----------------------+
|codigo_ibge|area_do_imovel|area_remanescente_vegetacao_nativa|percentual_area_nativa|
+-----------+--------------+----------------------------------+----------------------+
|    1200302|       41.0618|                         16.521088|                 40.23|
|    1200252|        3.5671|                          2.423295|                 67.93|
|    1200401|       85.7813|                          14.51889|                 16.93|
|    1200104|       33.6201|                         16.234642|                 48.29|
|    1200450|       80.6683|                               0.0|                  0.00|
|    1200385|       44.5476|                         19.857813|                 44.58|
|    1200609|       42.9766|                         41.673923|                 96.97|
|    1200401|        0.9943|                        0.39500403|                 39.73|
|    1200401|       28.8663|               

0.60634446144104

## Consulta 6
Construa uma consulta que mostre a contagem de propriedades rurais por estado.

* AST – Assentamentos
* CAR – Cadastro Ambiental Rural
* CCU – Contrato de Concessão de Uso
* CDRU – Concessão de Direito Real de Uso
* DD – Diretoria de Desenvolvimento
* IRU – Imóveis Rurais
* MLC – Módulo Lote CAR
* PA – Projeto de Assentamento
* PCT – Povos e Comunidades Tradicionais
* PRA – Programa de Regularização Ambiental
* PNRA – Programa Nacional de Reforma Agrária
* SICAR – Sistema de Cadastro Ambiental Rural
* SR – Superintendência Regional
* TCMS – Termo de Compromisso de Manutenção de Sigilo
* TD – Título Definitivo
* TED – Termo de Execução Descentralizada
* URL – Uniform Resource Locator

In [9]:
query = """
SELECT
DISTINCT tipo_imovel_rural
                    
FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`
"""

delta_table_query(query)

+-----------------+
|tipo_imovel_rural|
+-----------------+
|              AST|
|              IRU|
|              PCT|
+-----------------+



In [10]:
query = """
SELECT
    uf,
    COUNT(tipo_imovel_rural) AS count_prop_rural

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

WHERE tipo_imovel_rural = 'IRU'

GROUP BY uf

ORDER BY count_prop_rural DESC
"""

tempo_delta_6 = get_time(delta_table_query, query)
tempo_delta_6

+---+----------------+
| uf|count_prop_rural|
+---+----------------+
| BA|         1011503|
| MG|          988921|
| RS|          606013|
| PR|          495650|
| SP|          406885|
| SC|          374768|
| PE|          338392|
| CE|          314168|
| PA|          278097|
| MA|          272010|
| PI|          254577|
| GO|          198524|
| PB|          170484|
| MT|          169775|
| RO|          147099|
| AL|          113222|
| ES|          108848|
| SE|           94203|
| RN|           88788|
| TO|           84990|
+---+----------------+
only showing top 20 rows



7.01518988609314

## Consulta 7
Veja qual é a maior propriedade entre todas e calcule a distância entre ela e Brasília. Utilize a coordenada de centródide da propriedade para calcular a distância entre ela e Brasília. Coordenadas de Brasília: -15.796943053171708, -47.891638482569476

In [11]:
query = """
SELECT
    max_area,
    latitude,
    longitude,
    FORMAT_NUMBER(
        SQRT(
            POWER(latitude - (-15.796943053171708), 2) + 
            POWER(longitude - (-47.891638482569476), 2) 
            )* 111, 2
        )  AS distancia_brasilia

FROM (

    SELECT 
        latitude, 
        longitude, 
        area_do_imovel AS max_area

    FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

    ORDER BY area_do_imovel DESC

    LIMIT 1
)
"""

tempo_delta_7 = get_time(delta_table_query, query)
tempo_delta_7

+---------+---------+---------+------------------+
| max_area| latitude|longitude|distancia_brasilia|
+---------+---------+---------+------------------+
|2420078.8|-5.475619|-68.88819|          2,596.98|
+---------+---------+---------+------------------+



6.444031476974487

## Consulta 8
Faça a média de área entre todas as propriedades. Calcule quantas propriedades por estado, estão acima da média.

In [12]:
query = """
WITH 
    avg_calc AS (
        SELECT
            AVG(area_do_imovel) AS avg_area

        FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

        GROUP BY uf
        ), 

    filtro_estado AS (
        SELECT
            uf,
            area_do_imovel

        FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`, avg_calc

        WHERE area_do_imovel > avg_calc.avg_area
)

SELECT
    uf,
    COUNT(*) AS acima_media

FROM filtro_estado

GROUP BY uf

ORDER BY acima_media DESC

"""

tempo_delta_8 = get_time(delta_table_query, query)
tempo_delta_8

+---+-----------+
| uf|acima_media|
+---+-----------+
| MG|    4659636|
| PA|    3040492|
| MT|    2423798|
| BA|    2301168|
| GO|    1991271|
| SP|    1942999|
| RS|    1930717|
| PR|    1668279|
| MA|    1628578|
| RO|    1337519|
| PI|    1263794|
| CE|    1043851|
| TO|     979080|
| MS|     957729|
| AM|     732345|
| SC|     729461|
| PE|     661795|
| AC|     492997|
| ES|     395320|
| PB|     385550|
+---+-----------+
only showing top 20 rows



13.55098843574524

## Tempo Total Delta Tables

In [13]:
total_delta = tempo_delta_1 + tempo_delta_2 + tempo_delta_3 + tempo_delta_4 + tempo_delta_5 + tempo_delta_6 + tempo_delta_7 + tempo_delta_8
total_delta

66.72776246070862

# Querys a partir da Hive Metastore

## Salvando Gold (Distribuída)

In [ ]:
# spark.sql("DROP TABLE IF EXISTS gold_distribuida_uf")

DataFrame[]

In [30]:
df_test = spark.read.format("delta").load("/home/jovyan/app/data/silver/silver_distribuida_uf")

df_test.write.partitionBy("uf").saveAsTable("gold_distribuida_uf")

## Consulta 1

Recupere a soma de área (em hectares) para todas as propriedades agrícolas que pertencem ao MS e MT. Ordene os resultados em ordem decrescente.

In [31]:
query = """
SELECT 
   uf,
   FORMAT_NUMBER(SUM(area_do_imovel), 3) as soma_area_hectares

FROM gold_distribuida_uf

WHERE uf IN ('MS', 'MT')

GROUP BY uf

ORDER BY soma_area_hectares DESC

"""

tempo_hive_1 = get_time(delta_table_query, query)
tempo_hive_1

+---+------------------+
| uf|soma_area_hectares|
+---+------------------+
| MT|    84,587,089.218|
| MS|    36,321,882.660|
+---+------------------+



0.33525586128234863

## Consulta 2
Filtre todas as propriedades que pertecem a região sudeste. 

In [32]:
query = """
SELECT
   uf,
   codigo_ibge

FROM gold_distribuida_uf

"""

tempo_hive_2 = get_time(delta_table_query, query)
tempo_hive_2

+---+-----------+
| uf|codigo_ibge|
+---+-----------+
| MG|    3136801|
| MG|    3136520|
| MG|    3145604|
| MG|    3171808|
| MG|    3104007|
| MG|    3123908|
| MG|    3114600|
| MG|    3155603|
| MG|    3171071|
| MG|    3112307|
| MG|    3112406|
| MG|    3169307|
| MG|    3108503|
| MG|    3139904|
| MG|    3119906|
| MG|    3170206|
| MG|    3165578|
| MG|    3158201|
| MG|    3104452|
| MG|    3131604|
+---+-----------+
only showing top 20 rows



1.0503020286560059

## Consulta 3
Recupere todas as propriedades rurais que estão localizadas dentro de uma área geográfica específica delimitada por um polígono. Este polígono é descrito pelas seguintes coordenadas: POLYGON ((-53.5325072 -19.4632582, -51.0495971 -19.1625841, -51.3734501 -16.1924262, -53.8181518 -16.4010783, -53.5325072 -19.4632582))
Verificar se um centroide está dentro dos pontos informados (ou mais).

In [33]:
query = """
SELECT
    codigo_ibge,
    latitude,
    longitude,
    is_point_in_polygon(latitude, longitude) AS inside_polygon

FROM gold_distribuida_uf

WHERE is_point_in_polygon(latitude, longitude) = True
"""

tempo_hive_3 = get_time(delta_table_query, query)
tempo_hive_3

+-----------+--------+---------+--------------+
|codigo_ibge|latitude|longitude|inside_polygon|
+-----------+--------+---------+--------------+
+-----------+--------+---------+--------------+



11.994521379470825

## Consulta 4
Calcule quantas propriedades foram cadastradas por ano. Apresente os resultados em ordem cronológica.

In [34]:
query = """
SELECT
   YEAR(data_inscricao) AS ano_inscricao,
   COUNT(*) AS qnt_ano

FROM gold_distribuida_uf

GROUP BY ano_inscricao

ORDER BY ano_inscricao
"""

tempo_hive_4 = get_time(delta_table_query, query)
tempo_hive_4

+-------------+-------+
|ano_inscricao|qnt_ano|
+-------------+-------+
|         NULL|     12|
|         2013|    743|
|         2014| 227468|
|         2015|1240176|
|         2016|1828786|
|         2017| 794777|
|         2018| 724856|
|         2019| 746570|
|         2020| 463322|
|         2021| 443942|
|         2022| 368448|
+-------------+-------+



1.716068983078003

## Consulta 5
Calcule o percentual médio de área remanescente de vegetação nativa em comparação a área total da propriedade

In [35]:
query = """
SELECT
   codigo_ibge,
   area_do_imovel,
   area_remanescente_vegetacao_nativa,
   FORMAT_NUMBER(area_remanescente_vegetacao_nativa / area_do_imovel * 100, 2) as percentual_area_nativa

FROM gold_distribuida_uf
"""

tempo_hive_5 = get_time(delta_table_query, query)
tempo_hive_5

+-----------+--------------+----------------------------------+----------------------+
|codigo_ibge|area_do_imovel|area_remanescente_vegetacao_nativa|percentual_area_nativa|
+-----------+--------------+----------------------------------+----------------------+
|    3136801|      119.0454|                               0.0|                  0.00|
|    3136520|       34.5053|                               0.0|                  0.00|
|    3145604|        3.5613|                               0.0|                  0.00|
|    3171808|       22.0177|                         1.1560909|                  5.25|
|    3104007|       25.9163|                         5.0354385|                 19.43|
|    3123908|        48.444|                               0.0|                  0.00|
|    3114600|       35.5403|                         7.0675855|                 19.89|
|    3155603|        1.8732|                               0.0|                  0.00|
|    3171071|       20.7036|               

0.17835283279418945

## Consulta 6
Construa uma consulta que mostre a contagem de propriedades rurais por estado.

In [36]:
query = """
SELECT
    uf,
    COUNT(tipo_imovel_rural) AS count_prop_rural

FROM gold_distribuida_uf

WHERE tipo_imovel_rural = 'IRU'

GROUP BY uf

ORDER BY count_prop_rural DESC
"""

tempo_hive_6 = get_time(delta_table_query, query)
tempo_hive_6

+---+----------------+
| uf|count_prop_rural|
+---+----------------+
| BA|         1011503|
| MG|          988921|
| RS|          606013|
| PR|          495650|
| SP|          406885|
| SC|          374768|
| PE|          338392|
| CE|          314168|
| PA|          278097|
| MA|          272010|
| PI|          254577|
| GO|          198524|
| PB|          170484|
| MT|          169775|
| RO|          147099|
| AL|          113222|
| ES|          108848|
| SE|           94203|
| RN|           88788|
| TO|           84990|
+---+----------------+
only showing top 20 rows



1.3295278549194336

## Consulta 7
Veja qual é a maior propriedade entre todas e calcule a distância entre ela e Brasília. Utilize a coordenada de centródide da propriedade para calcular a distância entre ela e Brasília. Coordenadas de Brasília: -15.796943053171708, -47.891638482569476

In [37]:
query = """
SELECT
    max_area,
    latitude,
    longitude,
    FORMAT_NUMBER(
        SQRT(
            POWER(latitude - (-15.796943053171708), 2) + 
            POWER(longitude - (-47.891638482569476), 2) 
            )* 111, 2
        )  AS distancia_brasilia

FROM (

    SELECT 
        latitude, 
        longitude, 
        area_do_imovel AS max_area

    FROM gold_distribuida_uf

    ORDER BY area_do_imovel DESC

    LIMIT 1
)
"""

tempo_hive_7 = get_time(delta_table_query, query)
tempo_hive_7

+---------+---------+---------+------------------+
| max_area| latitude|longitude|distancia_brasilia|
+---------+---------+---------+------------------+
|2420078.8|-5.475619|-68.88819|          2,596.98|
+---------+---------+---------+------------------+



1.5660717487335205

## Consulta 8
Faça a média de área entre todas as propriedades. Calcule quantas propriedades por estado, estão acima da média.

In [38]:
query = """
WITH 
    avg_calc AS (
        SELECT
            AVG(area_do_imovel) AS avg_area

        FROM gold_distribuida_uf

        GROUP BY uf
        ), 

    filtro_estado AS (
        SELECT
            uf,
            area_do_imovel

        FROM gold_distribuida_uf, avg_calc

        WHERE area_do_imovel > avg_calc.avg_area
)

SELECT
    uf,
    COUNT(*) AS acima_media

FROM filtro_estado

GROUP BY uf

ORDER BY acima_media DESC

"""

tempo_hive_8 = get_time(delta_table_query, query)
tempo_hive_8

+---+-----------+
| uf|acima_media|
+---+-----------+
| MG|    4659636|
| PA|    3040492|
| MT|    2423798|
| BA|    2301168|
| GO|    1991271|
| SP|    1942999|
| RS|    1930717|
| PR|    1668279|
| MA|    1628578|
| RO|    1337519|
| PI|    1263794|
| CE|    1043851|
| TO|     979080|
| MS|     957729|
| AM|     732345|
| SC|     729461|
| PE|     661795|
| AC|     492997|
| ES|     395320|
| PB|     385550|
+---+-----------+
only showing top 20 rows



3.3028199672698975

## Tempo Total Hive Metastore

In [39]:
total_hive = tempo_hive_1 + tempo_hive_2 + tempo_hive_3 + tempo_hive_4 + tempo_hive_5 + tempo_hive_6 + tempo_hive_7 + tempo_hive_8
total_hive

21.472920656204224

# Querys a partir de Banco Centralizado (Professor)